In [1]:
import spacy
nlp = spacy.load('en_core_web_lg', disable=["parser", "tagger", "ner"])

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

In [3]:
fun_fact_df = pd.read_csv('../data/fun_fact_title.csv')

/Users/danny/Documents/College-CORNELL/04 Senior-MEng/Spring 2019/INFO4300 - Language and Information/final project/fun-fax/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,8,10,11,13,14,15,16,17,19,20,21,22,23,24,25,26,28,29,30,31,33,37,38,43,44,47,48,49,50,51,52,54,55,57,58,60,61,62,64,65,71,75,87,89,91,92,93,94,97,101,104,105,106,107,110,111,112,113,114,115,118,119,120,121,122,129,133,135,137,138,139,140,142,143,148,152,155,174,176,178,179,180,181,187,191,192,193,194,197,199,201,202,203,209) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
required_columns = ['title', 'subreddit', 'permalink']
fun_fact_df = fun_fact_df.dropna(axis='rows', subset=required_columns).reset_index(drop=True)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy

In [7]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=.8, ngram_range=(1,1))
titles = fun_fact_df['title']
fun_fact_tfidf = vectorizer.fit_transform(titles)

In [8]:
fun_fact_tfidf

<39148x35740 sparse matrix of type '<class 'numpy.float64'>'
	with 249410 stored elements in Compressed Sparse Row format>

In [11]:
assert "fun" not in vectorizer.get_feature_names()

In [12]:
len(vectorizer.get_feature_names())

35740

In [13]:
len(fun_fact_df['title'])

39148

In [14]:
features = vectorizer.get_feature_names()
weighted_embedding = np.zeros((len(titles), 300))
for row, col in zip(*fun_fact_tfidf.nonzero()):
    weighted_embedding[row] += nlp.vocab[features[col]].vector * fun_fact_tfidf[row,col]

In [15]:
avg_weighted = np.array(weighted_embedding) #/ (np.sum(fun_fact_tfidf, axis=1) + .00001))

In [16]:
np.sum(fun_fact_tfidf > 0, axis=1) # shows number of tfidf words per doc

matrix([[1],
        [0],
        [0],
        ...,
        [7],
        [5],
        [5]])

In [17]:
print(np.linalg.norm(avg_weighted[22291]))
print(np.linalg.norm(avg_weighted[10493]))

6.96032559561319
6.9589698099915065


In [18]:
query = "donald trump"
query_tfidf = vectorizer.transform([query])
query_weighted = np.zeros(300)
for row, col in zip(*query_tfidf.nonzero()):
    query_weighted += nlp.vocab[features[col]].vector * query_tfidf[row,col]
rankings = avg_weighted.dot(query_weighted)
fun_fact_df["title"][np.argsort(-rankings)]

37017    Fun Fact: Last season Kemba Walker became one of five point guards in NBA history to be a franchise’s all-time leading scorer joining Isiah Thomas (Detroit Pistons), Hal Greer (Philadelphia 76ers), Oscar Robertson (Sacramento Kings), and Gary Payton (Seattle SuperSonics/Oklahoma City Thunder).     
33302    Fun Fact: After 16 games, Lonzo Ball is shooting better from 3 point distance than Klay Thompson, Kevin Durant, Bradley Beal, John Wall, Kawhi Leonard, Devin Booker, Donovan Mitchell, Jamal Murray, and Trae Young.                                                                                      
32182    Fun Fact: After 16 games, Lonzo Ball is shooting better from 3 point distance than Klay Thompson, Kevin Durant, Bradley Beal, John Wall, Kawhi Leonard, Devin Booker, Donovan Mitchell, Jamal Murray, and Trae Young.                                                                                      
29390    Fun fact: Aristotle, Genghis Khan, Isaac Newton, Beethoven, Mari

In [19]:
query_tfidf.data

array([0.63616713, 0.77155128])

In [20]:
rankings[np.argsort(-rankings)][:5]

array([73.46073664, 69.2093255 , 69.2093255 , 69.07042532, 67.06257083])